In [1]:
import pandas as pd
import numpy
from collections import defaultdict

In [2]:
bdscom = pd.read_csv("bdscom.csv")

In [3]:
address = pd.read_excel("/Users/thaochie/Documents/hanoiaddress.xlsx")

In [4]:
address["Quận Huyện"]=address["Quận Huyện"].apply(lambda x: x.replace("Quận ","").strip().lower())
address["Quận Huyện"]=address["Quận Huyện"].apply(lambda x: x.replace("huyện ","").strip().lower())
address["Quận Huyện"]

0      ba đình
1      ba đình
2      ba đình
3      ba đình
4      ba đình
        ...   
574     mỹ đức
575     mỹ đức
576     mỹ đức
577     mỹ đức
578     mỹ đức
Name: Quận Huyện, Length: 579, dtype: object

In [5]:
address["Xã Phường"]=address["Xã Phường"].apply(lambda x: x.lower().replace("phường ","").strip().lower())
address["Xã Phường"]=address["Xã Phường"].apply(lambda x: x.lower().replace("xã ","").strip().lower())

address["Xã Phường"]

0        phúc xá
1      trúc bạch
2      vĩnh phúc
3        cống vị
4      liễu giai
         ...    
574    hùng tiến
575      an tiến
576     hợp tiến
577    hợp thanh
578       an phú
Name: Xã Phường, Length: 579, dtype: object

In [6]:
districts = set(address["Quận Huyện"].tolist())
s = defaultdict(int)
for idx, i in enumerate(bdscom["District"]):
    if i.lower() not in districts and i.lower() not in s:
        print(i)
        if i == "None":
            continue
        s[i.lower()] += 1
        bdscom = bdscom[bdscom["District"] != i]
bdscom.head()

Sơn Tây
Quận Đống Đa
Quận Tây Hồ
Quận Nam Từ Liêm
Huyện Gia Lâm
Huyện Thanh Trì
nam từ liêm
Huyện Chương Mỹ
Quận Hai Bà Trưng
Huyện Hoài Đức
Quận Hoàng Mai
Huyện Đông Anh
Quận Long Biên
Huyện Mê Linh
Quận Ba Đình
Quận Hà Đông
Quận Thanh Xuân
Quận Cầu Giấy
quận Đan Phượng
phường Bách Khoa
Quận Bắc Từ Liêm
Huyện Thạch Thất
Huyện Đan Phượng
Huyện Quốc Oai
Từ Liêm


,House Direction,Balcony Direction,Bedrooms,Toilets,Legits,Furnitures,Floors,Facade,Entrance,City,District,Ward,Street,Area,Price
0,Tây - Nam,Đông - Bắc,1.0,1.0,Hợp đồng mua bán,NaN,NaN,NaN,NaN,Hà Nội,Tây Hồ,Phường Phú Thượng,Đường Lạc Long Quân,45.25,2715.0
1,NaN,Tây,2.0,2.0,Sổ đỏ/ Sổ hồng.,"- Hoàn thiện nội thất cơ bản CĐT, đã làm full ...",NaN,NaN,NaN,Hà Nội.,Nam Từ Liêm,Phường Mỹ Đình 1,Đường Phạm Hùng,70.00,3700.0
2,NaN,NaN,10.0,10.0,NaN,Full nội thất.,6.0,6.5,2.5,Hà Nội,Hai Bà Trưng,Phường Thanh Lương,Phố Lạc Trung,60.00,9300.0
3,NaN,NaN,2.0,2.0,Đang chờ sổ,Đầy đủ,NaN,NaN,NaN,Hà Nội,Tây Hồ,Phường Phú Thượng,Đường Lạc Long Quân,60.00,3600.0
4,NaN,NaN,NaN,NaN,Sổ đỏ/ Sổ hồng,NaN,5.0,4.5,NaN,Hà Nội,Cầu Giấy,Phường Trung Hòa,Đường Nguyễn Thị Định,65.00,15000.0


CRAWL GEO COORDINATES

In [7]:
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm

def get_loc(x, lat=True):
    while True:
        try:
            x.replace("None", "")
            loc = Nominatim(user_agent="GetLoc")
            getLoc = loc.geocode(x)
            if getLoc is None:
                loc = Nominatim(user_agent="GetLoc")
                getLoc = loc.geocode("Hà Nội")
            return (  getLoc.latitude, getLoc.longitude)
        except:
            time.sleep(0.5)
    

In [8]:
address = bdscom["Ward"] + " " + bdscom["District"] + " hà nội"

bdscom["X"] = [0 for i in address]
bdscom["Y"] = [0 for i in address]

x = [0 for i in address]
y = [0 for i in address]

In [9]:
import time
from tqdm import tqdm
import pandas as pd

# Giả sử lookups đã khởi tạo
lookups = {} 

for idx, i in enumerate(tqdm(address)):
    try:
        # 1. Kiểm tra dữ liệu đầu vào
        if pd.isna(i) or i == "":
            print(f"\n[SKIP] Dòng {idx} bị rỗng")
            continue

        if i in lookups:
            x[idx] = lookups[i][0]
            y[idx] = lookups[i][1]
        else:
            # 2. Gọi hàm lấy toạ độ
            loc_result = get_loc(i)
            
            # 3. Kiểm tra kết quả trả về có hợp lệ không
            # Giả sử get_loc trả về [lat, long] hoặc None/[] nếu lỗi
            if loc_result and len(loc_result) >= 2:
                lookups[i] = loc_result
                x[idx] = loc_result[0]
                y[idx] = loc_result[1]
            else:
                print(f"\n[FAIL] Dòng {idx}: Không tìm thấy toạ độ cho '{i}'")
                x[idx] = None
                y[idx] = None
            
            # 4. QUAN TRỌNG: Ngủ 1 chút để không bị server chặn (1 giây)
            time.sleep(1) 

    except Exception as e:
        # 5. Bắt mọi lỗi bất ngờ để vòng lặp không bao giờ chết
        print(f"\n[ERROR] Lỗi tại dòng {idx} ({i}): {e}")
        x[idx] = None
        y[idx] = None

    # Lưu file định kỳ
    if idx % 20 == 0 or idx == len(address) - 1:
        bdscom["X"] = x
        bdscom["Y"] = y
        bdscom.to_csv("final.csv", index=False)

  1%|          | 139/21228 [02:47<4:33:35,  1.28it/s] 


[SKIP] Dòng 139 bị rỗng


  1%|          | 147/21228 [02:56<5:30:29,  1.06it/s]


[SKIP] Dòng 147 bị rỗng


  1%|          | 151/21228 [02:57<4:13:55,  1.38it/s]

...

 99%|█████████▊| 20961/21228 [21:40<00:04, 59.01it/s]


[SKIP] Dòng 20974 bị rỗng


 99%|█████████▉| 21121/21228 [21:42<00:01, 67.98it/s]


[SKIP] Dòng 21130 bị rỗng

[SKIP] Dòng 21131 bị rỗng


100%|██████████| 21228/21228 [21:46<00:00, 16.25it/s]
